## Time Series Forecasting Workshop, Sep 2022

*NUS MSBA 2022, Sept 17th*

*By Shivam Bansal*

---


### Run this Notebook

**Option1:**

- Preferred for Mac / Linux Users. 

- Directly in your Laptop: This notebook can be run directly in your laptop, but need to ensure that required packages are installed. 

**Option2:**

- Run in Google Collab. (Better Option for Windows Users)

   [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/masternotebooks/colab-github-demo.ipynb)


### Install All Packages 




In [ ]:
#### Uncomment the following line to install all the required packages used in this notebook ####

# !pip3 install matplotlib pandas numpy seaborn statsmodels sklearn xgboost scipy pmdarima prophet

### 1. Import the Required Packages

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.holtwinters import Holt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.api import SimpleExpSmoothing, ExponentialSmoothing
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.interpolate import interp1d
import xgboost as xgb
import pmdarima as pm
from prophet import Prophet

import matplotlib.pyplot as plt 
from pandas import Series
import seaborn as sns 
import pandas as pd 
import numpy as np

pd.options.mode.chained_assignment = None


### 2. Load Dataset and PreProcessing

Monthly Passengers Data from 1949 to 1960 

In [ ]:
#### Uncomment and Add Code ####
# df = pd.read_csv("")
print ("df Shape: ", df.shape)

#### Uncomment and Add Code ####
# date_column, target_column = '', ''

print ("df Sample:")
df.head()

In [ ]:
## 2.1 Convert date_column into proper datetime format 

#### Uncomment and Add Code ####
# df[date_column] = <FUNCTION>(df[date_column])

## 2.2 Visualize the dataset 
def line_chart(x_col, y_col, df, title = ""):
    plt.figure(figsize=(15, 6))
    plt.title(title);
    sns.lineplot(x = x_col, y = y_col, data = df, color = 'green')
    plt.show();
    
line_chart(x_col = date_column, y_col = target_column, df = df, title = "Monthly Airline Passengers")

## 2.3 Split the dataset - Training and Test Set
train_df, test_df = df[:108], df[108:]
print ("Training Set Shape: ", train_df.shape)
print ("Test Set Shape: ", test_df.shape)

In [ ]:
## 2.4 Missing Values - Detection and Imputation Techniques 

df_o = df.copy(deep = True)
df_o = df_o.set_index(date_column, inplace = False)

df_m = pd.read_csv('data/passengers_with_missing.csv')
df_m[date_column] = pd.to_datetime(df_m[date_column], infer_datetime_format = True)
df_m = df_m.set_index(date_column, inplace = False)

fig, axes = plt.subplots(5, 1, sharex=True, figsize=(15, 18))
plt.rcParams.update({'xtick.bottom' : False})

## 1. Actual 
df_o.plot(title='Actual', ax=axes[0], label='Actual', color='red', style=".-")
df_m.plot(title='Actual', ax=axes[0], label='Actual', color='green', style=".-")
axes[0].legend(["Missing Data", "Available Data"])

## 2. Forward Fill 
# df_ffill = df_m.ffill()
# df_ffill[target_column].plot(title='Forward Fill', ax=axes[1], label='Forward Fill', style=".-")

## 3. Backward Fill 
# df_bfill = df_m.bfill()
# df_bfill[target_column].plot(title="Backward Fill", ax=axes[2], label='Backward Fill', color='firebrick', style=".-")

## 4. Linear Interpolation 
# df_m['row_num'] = np.arange(df_o.shape[0])
# df_nona = df_m.dropna(subset = [target_column])
# f = interp1d(df_nona['row_num'], df_nona[target_column])
# df_m['linear_fill'] = f(df_m['row_num'])
# df_m['linear_fill'].plot(title="Linear Fill", ax=axes[3], label='Cubic Fill', color='brown', style=".-")

## 5. Cubic Interpolation 
# f2 = interp1d(df_nona['row_num'], df_nona[target_column], kind='cubic')
# df_m['cubic_fill'] = f2(df_m['row_num'])
# df_m['cubic_fill'].plot(title="Cubic Fill", ax=axes[4], label='Cubic Fill', color='red', style=".-")

### 3. Decomposition - Time Series Components 

Additive: 
y(t) = Level + Trend + Seasonality + Noise

Multiplicative: 
y(t) = Level * Trend * Seasonality * Noise

In [ ]:
#### Uncomment and Add Code ####
# series = list(df[target_column])
# result = seasonal_decompose(series, model='', period=?)

fig = result.plot()
fig.set_size_inches((14, 8))
fig.tight_layout()
plt.show()

In [ ]:
#### Uncomment and Add Code ####
# series = list(df[target_column])
# result = seasonal_decompose(series, model='', period=?)

fig = result.plot()
fig.set_size_inches((14, 8))
fig.tight_layout()
plt.show()

### 4. Model - Time Series Forecasting 

### 4.1 Model1: Simple Moving Average 

In [ ]:
#### Uncomment and Add Code ####
# window = ?
# rolling_mean = df[target_column].rolling(window=window).mean()

plt.figure(figsize=(15,5))
plt.title("Moving Average, window size = " + str(window))
plt.plot(rolling_mean, "g", label="Rolling mean trend")
plt.plot(series[window:], label="Actual values")
plt.legend(loc="upper left")
plt.grid(True)

In [ ]:
## Prediction at T = Mean (t-1, t-2, t-3, ... t-n)

mean_val = 0
for i in range(window):
    mean_val += df[target_column].iloc(0)[108-i] 
    
print (mean_val / window)

### 4.2 Model2: Simple Exponential Smoothing

Weighted averages - Recent observation has more weight than past observations. 

Exponential smoothing forecasting methods are similar in that a prediction is a weighted sum of past observations, but the model explicitly uses an exponentially decreasing weight for past observations.


smoothing parameter: α
- When α = 0, the forecasts are equal to the average of the historical data 
- When α = 1, the forecasts will be equal to the value of the last observation

Large values mean that the model pays attention mainly to the most recent past observations, whereas smaller values mean more of the history is taken into account when making a prediction.

In [ ]:
plt.figure(figsize=(18, 8))
plt.plot(train_df[target_column], marker='o', color="black")

fit1 = SimpleExpSmoothing(train_df[target_column]).fit(smoothing_level=0.1, optimized=False)
plt.plot(fit1.fittedvalues, marker="o", color="b")

fit2 = SimpleExpSmoothing(train_df[target_column]).fit(smoothing_level=0.5, optimized=False)
plt.plot(fit2.fittedvalues, marker="o", color="r")

fit3 = SimpleExpSmoothing(train_df[target_column]).fit(smoothing_level=1, optimized=False)
plt.plot(fit3.fittedvalues, marker="o", color="g")

plt.xticks(rotation="vertical")
plt.show()

In [ ]:
fit1.predict()    

### Model3: 4.3 Holt Linear

Expanding the SES method, the Holt method helps you forecast time series data that has a trend. In addition to the level smoothing parameter α introduced with the SES method, the Holt method adds the trend smoothing parameter β*. Like with parameter α, the range of β* is also between 0 and 1.

In [ ]:
plt.figure(figsize=(18, 8))
plt.plot(train_df[target_column], marker='o', color="black")

fit1 = ExponentialSmoothing(train_df[target_column]).fit(smoothing_level=0.1, smoothing_slope=0.1, optimized=False)
plt.plot(fit1.fittedvalues, marker="o", color="b")

fit2 = ExponentialSmoothing(train_df[target_column]).fit(smoothing_level=0.5, smoothing_slope=0.5, optimized=False)
plt.plot(fit2.fittedvalues, marker="o", color="r")

fit3 = ExponentialSmoothing(train_df[target_column]).fit(smoothing_level=1, smoothing_slope=0.1, optimized=False)
plt.plot(fit3.fittedvalues, marker="o", color="g")

plt.xticks(rotation="vertical")
plt.show()

In [ ]:
fit1 = Holt(train_df[target_column]).fit()  
fit2 = Holt(train_df[target_column], exponential=True).fit() 

plt.figure(figsize=(18, 8))
plt.plot(train_df[target_column], marker='o', color='black')
plt.plot(fit1.fittedvalues, marker='o', color='b')
plt.plot(fit2.fittedvalues, marker='o', color='r')
plt.xticks(rotation="vertical")
plt.show()

In [ ]:
fit1.predict()    

### Model4: 4.4 Holt Winter with Seasonal Effect

In [ ]:
def holt_win_sea(y,y_to_train,y_to_test,seasonal_type,seasonal_period,predict_date):
    
    y.plot(marker='o', color='black', legend=True, figsize=(14, 7))
    
    if seasonal_type == 'additive':
        fit1 = ExponentialSmoothing(y_to_train, seasonal_periods = seasonal_period, trend='add', seasonal='add').fit()
        fcast1 = fit1.forecast(predict_date).rename('Additive')
        mse1 = ((fcast1 - y_to_test) ** 2).mean()
        print('The RMSE of additive trend, additive seasonal of period season_length={}'.format(seasonal_period))
        
        fit2 = ExponentialSmoothing(y_to_train, seasonal_periods = seasonal_period, trend='add', seasonal='add', damped=True).fit()
        fcast2 = fit2.forecast(predict_date).rename('Additive+damped')
        mse2 = ((fcast2 - y_to_test) ** 2).mean()
        print('The RMSE of additive damped trend, additive seasonal of period season_length={}'.format(seasonal_period))

        fit1.fittedvalues.plot(style='--', color='red')
        fcast1.plot(style='--', marker='o', color='red', legend=True)
        fit2.fittedvalues.plot(style='--', color='green')
        fcast2.plot(style='--', marker='o', color='green', legend=True)
    
    elif seasonal_type == 'multiplicative':  
        fit3 = ExponentialSmoothing(y_to_train, seasonal_periods = seasonal_period, trend='add', seasonal='mul').fit()
        fcast3 = fit3.forecast(predict_date).rename('Multiplicative')
        mse3 = ((fcast3 - y_to_test) ** 2).mean()
        print('The RMSE of multiplicative trend, multiplicative seasonal of period season_length={}'.format(seasonal_period))
        
        fit4 = ExponentialSmoothing(y_to_train, seasonal_periods = seasonal_period, trend='add', seasonal='mul', damped=True).fit()
        fcast4 = fit4.forecast(predict_date).rename('Multiplicative+damped')
        mse4 = ((fcast3 - y_to_test) ** 2).mean()
        print('The RMSE of multiplicative trend, multiplicative damped seasonal of period season_length={}'.format(seasonal_period))
        
        fit3.fittedvalues.plot(style='--', color='red')
        fcast3.plot(style='--', marker='o', color='red', legend=True)
        fit4.fittedvalues.plot(style='--', color='green')
        fcast4.plot(style='--', marker='o', color='green', legend=True)
        
    else:
        print('Wrong Seasonal Type. Please choose between additive and multiplicative')

    plt.show()

y = df[target_column]
train_y = train_df[target_column]
test_y = test_df[target_column]
predict_date = len(y) - len(test_y)
    
#### Uncomment and Add Code ####
# holt_win_sea(y, train_y, test_y,'?', ?, predict_date)
# holt_win_sea(y, train_y, test_y,'?', ?, predict_date)


### Model #5: ARIMA 

Auto Regressive Integrated with Moving Average 

---

- AR(p) Autoregression – a regression model that utilizes the dependent relationship between a current observation and observations over a previous period. An auto regressive (AR(p)) component refers to the use of past values in the regression equation for the time series

- I(d) Integration – uses differencing of observations (subtracting an observation from observation at the previous time step) in order to make the time series stationary. Differencing involves the subtraction of the current values of a series with its previous values d number of times

- MA(q) Moving Average – a model that uses the dependency between an observation and a residual error from a moving average model applied to lagged observations. A moving average component depicts the error of the model as a combination of previous error terms. The order q represents the number of terms to be included in the model

---

- p is the order of the AR term
- q is the order of the MA term
- d is the differencing steps required to make the time series stationary


In [ ]:
from statsmodels.tsa.arima.model import ARIMA

#### Uncomment and Add Code ####
# model4 = ARIMA(train_df[target_column].values, order=(?, ?, ?))
model4 = model4.fit()
print (model4.summary())

In [ ]:
residuals = pd.DataFrame(model4.resid)
fig, ax = plt.subplots(1,2, figsize=(15, 4))
residuals.plot(title="Residuals", ax=ax[0])
residuals.plot(kind='kde', title='Density', ax=ax[1])
plt.show()

### Auto ARIMA

In [ ]:
# more details here.- https://alkaline-ml.com/pmdarima/modules/generated/pmdarima.arima.auto_arima.html

model4_1 = pm.auto_arima(train_df[target_column].values, start_p=1, start_q=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

print(model4_1.summary())

In [ ]:
model4_1.plot_diagnostics(figsize=(10,8))
plt.show()

In [ ]:
#### Uncomment and Add Code ####
# model4_2 = ARIMA(train_df[target_column].values, order=(?,?,?))
model4_2 = model4_2.fit()
print(model4.summary())

In [ ]:
start = len(train_df)
end  = len(train_df) + len(test_df) -1

test_pred = model4_2.predict(start=start, end=end, dynamic=False, typ="levels")
test_df['pred'] = test_pred

In [ ]:
mae = mean_absolute_error(test_df[target_column], test_df["pred"])
print('ARIMA Model Test MAE: %.3f' % mae)

plt.figure(figsize=(15, 8))
plt.title("Model - ARIMA - Time Series Forecasting");
sns.lineplot(x = date_column, y = target_column, data = df, color = 'green', label='Actual')
sns.lineplot(x = date_column, y = 'pred', data = test_df, color = 'red', label = 'Test Predictions')
plt.show();

### Model #6: Facebook Prophet

1. Open Source time series forecasting model developed by Facebook
2. Based on an Additive model where non-linear trends are fit with yearly, weekly, and daily seasonality, plus holiday effects. 
3. Works best with time series that have strong seasonal effects and several seasons of historical data
4. Prophet is robust to missing data and shifts in the trend, and typically handles outliers well.
5. Mainly built for univariate time series models . though there are extensions avialable 


In [ ]:
## prepare data for prophet model 
data_for_prophet = train_df[[date_column, target_column]].rename(columns={date_column: "ds", target_column: "y"})
data_for_prophet_test = test_df[[date_column, target_column]].rename(columns={date_column: "ds"})

## fit the prophet model 
model5 = Prophet()
model5.fit(data_for_prophet)

## predict on train 
forecast = model5.predict(data_for_prophet)
train_pred = forecast.yhat.values
train_df['pred'] = train_pred

## predict on test
forecast = model5.predict(data_for_prophet_test)
test_pred = forecast.yhat.values
test_df['pred'] = test_pred

## Evaluate Model Performance 
mae = mean_absolute_error(train_df[target_column], train_df["pred"])
print('Prophet Model Train MAE: %.3f' % mae)

mae = mean_absolute_error(test_df[target_column], test_df["pred"])
print('Prophet Model Test MAE: %.3f' % mae)

plt.figure(figsize=(15, 8))
plt.title("Model - Prophet - Time Series Forecasting");
sns.lineplot(x = date_column, y = target_column, data = df, color = 'green', label='Actual')
sns.lineplot(x = date_column, y = 'pred', data = train_df, color = 'blue', label='Train Predictions')
sns.lineplot(x = date_column, y = 'pred', data = test_df, color = 'red', label = 'Test Predictions')
plt.show();

### Model #7: Machine Learning Approach 

Time Series forecasting problems can be structured as a machine learning (regression) task by adding elements / features of time series. 

In [ ]:
energy_hourly = pd.read_csv('data/energy_consumption.csv', index_col=[0], parse_dates=[0])
energy_hourly.sort_index(inplace=True)
target_column = 'PJMW_MW'

def split_data(data, split_date):
    return data[data.index <= split_date].copy(), \
           data[data.index >  split_date].copy()

train, test = split_data(energy_hourly, '01-Jan-2015')

plt.figure(figsize=(15,5))
plt.xlabel('time')
plt.ylabel('energy consumed')
plt.plot(train.index, train[target_column])
plt.plot(test.index, test[target_column])
plt.show()

In [ ]:
def create_features(df):
    """
    Creates time series features from datetime index
    """
    df['date']       = df.index
    df['hour']       = df['date'].dt.hour
    df['dayofweek']  = df['date'].dt.dayofweek
    df['quarter']    = df['date'].dt.quarter
    df['month']      = df['date'].dt.month
    df['year']       = df['date'].dt.year
    df['dayofyear']  = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.weekofyear
    
    X = df[['hour','dayofweek','quarter','month','year','dayofyear','dayofmonth','weekofyear']]
    return X

In [ ]:
X_train, y_train = create_features(train), train[target_column]
X_test, y_test   = create_features(test), test[target_column]

X_train.shape, y_train.shape

In [ ]:
reg = xgb.XGBRegressor(n_estimators=1000)

reg.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        early_stopping_rounds=50, 
        verbose=False) 

In [ ]:
X_test_pred = reg.predict(X_test)
mae = mean_absolute_error(y_test, X_test_pred)
print('xgBoost Model Test MAE: %.3f' % mae)

In [ ]:
plt.figure(figsize = (15, 5))
plt.xlabel('time')
plt.ylabel('energy consumed')
plt.plot(y_test.index, y_test)
plt.plot(X_test.index, X_test_pred)
plt.show()

### Model #8 - Deep Learning - LSTM

 
In this example we wish to make forcasts on a  time series of international airline passengers.
 
The time series data forcast can be modeled as a univariate regression-type problem, concretely let ${X_t}$ denote the number of airline passengers in month $t$. Then: 
 
$$
X_t = f(X_{t-1}, \Theta)
$$
 
which we aim to solve using the a simple LSTM neural network. 

Here $X_t$ is the number of passengers at time step $t$, $X_{t-1}$ denotes  number of passengers at the previous time step, and $\Theta$ refers to all the other model parameters, including LSTM hyperparameters.

*Note*: For better readability, in the code for this as well as the next example, the predicted new value at time step $t$ is written as `Y`. 

In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Original data set retrieved from here:
# https://datamarket.com/data/set/22u3/international-airline-passengers-monthly-totals-in-thousands-jan-49-dec-60#!ds=22u3&display=line

data = pd.read_csv("data/international-airline-passengers.csv", 
                      usecols = [1], 
                      engine = "python", 
                      skipfooter = 3)

Visualize data

In [ ]:
# Create a time series plot.
plt.figure(figsize = (15, 5))
plt.plot(data, label = "Airline Passengers")
plt.xlabel("Months")
plt.ylabel("1000 International Airline Passengers")
plt.title("Monthly Total Airline Passengers 1949 - 1960")
plt.legend()
plt.show()

Building LSTM Model

In [ ]:
# Let's load the required libs.
# We'll be using the Tensorflow backend (default).
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Activation, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle

In [ ]:
# Get the raw data values from the pandas data frame.
data_raw = data.values.astype("float32")

# We apply the MinMax scaler from sklearn
# to normalize data in the (0, 1) interval.
scaler = MinMaxScaler(feature_range = (0, 1))
dataset = scaler.fit_transform(data_raw)

# Print a few values.
dataset[0:5]

In [ ]:
# Using 60% of data for training, 40% for validation.
TRAIN_SIZE = 0.60

train_size = int(len(dataset) * TRAIN_SIZE)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size, :], dataset[train_size:len(dataset), :]
print("Number of entries (training set, test set): " + str((len(train), len(test))))

In [ ]:
# FIXME: This helper function should be rewritten using numpy's shift function. See below.
def create_dataset(dataset, window_size = 1):
    data_X, data_Y = [], []
    for i in range(len(dataset) - window_size - 1):
        a = dataset[i:(i + window_size), 0]
        data_X.append(a)
        data_Y.append(dataset[i + window_size, 0])
    return(np.array(data_X), np.array(data_Y))

In [ ]:
# Create test and training sets for one-step-ahead regression.
window_size = 1
train_X, train_Y = create_dataset(train, window_size)
test_X, test_Y = create_dataset(test, window_size)
print("Original training data shape:")
print(train_X.shape)

# Reshape the input data into appropriate form for Keras.
train_X = np.reshape(train_X, (train_X.shape[0], 1, train_X.shape[1]))
test_X = np.reshape(test_X, (test_X.shape[0], 1, test_X.shape[1]))
print("New training data shape:")
print(train_X.shape)

The LSTM architecture here consists of:

- One input layer.
- One LSTM layer of 4 blocks.
- One Dense layer to produce a single output.
- Use MSE as loss function.
- Many different architectures could be considered. But this is just a quick test, so we'll keep things nice and simple.

In [ ]:
def fit_model(train_X, train_Y, window_size = 1):
    model = Sequential()
    
    model.add(LSTM(4, 
                   input_shape = (1, window_size)))
    model.add(Dense(1))
    model.compile(loss = "mean_squared_error", 
                  optimizer = "adam")
    model.fit(train_X, 
              train_Y, 
              epochs = 100, 
              batch_size = 1, 
              verbose = 2)
    
    return(model)

# Fit the first model.
model1 = fit_model(train_X, train_Y, window_size)

In [ ]:
def predict_and_score(model, X, Y):
    # Make predictions on the original scale of the data.
    pred = scaler.inverse_transform(model.predict(X))
    # Prepare Y data to also be on the original scale for interpretability.
    orig_data = scaler.inverse_transform([Y])
    # Calculate RMSE.
    score = math.sqrt(mean_squared_error(orig_data[0], pred[:, 0]))
    return(score, pred)

rmse_train, train_predict = predict_and_score(model1, train_X, train_Y)
rmse_test, test_predict = predict_and_score(model1, test_X, test_Y)

print("Training data score: %.2f RMSE" % rmse_train)
print("Test data score: %.2f RMSE" % rmse_test)

In [ ]:
# Start with training predictions.
train_predict_plot = np.empty_like(dataset)
train_predict_plot[:, :] = np.nan
train_predict_plot[window_size:len(train_predict) + window_size, :] = train_predict

# Add test predictions.
test_predict_plot = np.empty_like(dataset)
test_predict_plot[:, :] = np.nan
test_predict_plot[len(train_predict) + (window_size * 2) + 1:len(dataset) - 1, :] = test_predict

# Create the plot.
plt.figure(figsize = (15, 5))
plt.plot(scaler.inverse_transform(dataset), label = "True value")
plt.plot(train_predict_plot, label = "Training set prediction")
plt.plot(test_predict_plot, label = "Test set prediction")
plt.xlabel("Months")
plt.ylabel("1000 International Airline Passengers")
plt.title("Comparison true vs. predicted training / test")
plt.legend()
plt.show()

## Thank You 

Reach out - Shivam Bansal (shivam.bansal@h2o.ai)